# Scrape Diamonds
Find web request URL that site is using to get data. This is found in network tab of developers tools. Only 20 rows at a time are possible. The `rowStart` steps 20 each iteration.

In [31]:
import pandas as pd
import requests

url = 'http://www.diamondse.info/webService.php?shape=none&minCarat=0.2&maxCarat=30&minColor=1'\
      '&maxColor=9&minPrice=100&maxPrice=100000&minCut=5&maxCut=1&minClarity=1&maxClarity=10'\
      '&minDepth=0&maxDepth=90&minWidth=0&maxWidth=90&gia=1&ags=1&egl=0&oth=0&currency=USD'\
      '&rowStart={}&sortCol=price&sortDir=DESC'

dfs = []
start = 0
step = 20
pages = 1000
stop = step * pages
for row in range(start, stop, step):
    r = requests.get(url.format(row))
    table = f"<table>{r.text}</table>"
    dfs.extend(pd.read_html(table))

In [32]:
# clean up

dia = pd.concat(dfs, ignore_index=True)
dia2 = dia.iloc[:, 1:11]
dia2.columns = ['shape', 'carat', 'cut', 'color', 'clarity', 'table', 'depth', 'cert', 'xyz', 'price']
dia2['price'] = dia2['price'].str.replace(r'[$,]', '').astype('int')
dia2[['x', 'y', 'z']] = dia2['xyz'].str.extract('([0-9.]*) x ([0-9.]*) x ([0-9.]*)').astype('float')
dia3 = dia2.drop(columns='xyz')
dia3.head()

,shape,carat,cut,color,clarity,table,depth,cert,price,x,y,z
0,Round,1.30,Ideal,F,SI1,57.0,62.3,GIA,9479,6.95,7.00,4.34
1,Cushion,2.02,V.Good,J,VS2,69.0,64.1,GIA,9479,8.46,6.72,4.30
2,Oval,1.50,Good,E,SI1,57.0,68.8,GIA,9479,8.65,6.08,4.18
3,Cushion,2.01,Ideal,I,VS2,61.0,69.2,GIA,9478,7.33,7.14,42.00
4,Round,1.36,Ideal,H,VS2,58.0,62.1,GIA,9478,7.10,7.07,4.40


In [33]:
dia3.shape

(20000, 12)